# Elaborato Open Data Management
# Davide Aloi

---

## Ontologia
### StackOverflowSurvery
Istanze del developer survey annuale di stack overflow. Caratterizzate da un anno di riferimento, un totale risposte e una lista di linguaggi di programmazione presentati come scelte.
Proprietà:
- hasLanguageOption
- totalResponses
- year
---
### SurveyRespondant
In base al campo "MainBranch" della risposta si distinguono tre sottoclassi disgiunte: Developer, LearningToCode, e NonDeveloper.
Considero anche qualcuno che ha lavorato come developer in passato un elemento della classe Developer.
Proprietà:
- maxAge
- minAge
- isFromCountry
- hasEmployment
- hasLanguageOption
- isFromSurvey
- usesLanguage
- usesOS
---
### Country
La nazione dove risiede l'utente. Ognuna ha associato il proprio codice ISO Alpha 3.
Le istanze di questa classe sono collegate a dbpedia con owl:sameAs.
Proprietà:
- ISO_A3
---
### EmploymentStatus
Rappresenta lo stato di carriera attuale di un utente. Le sue istanze definite insieme all'ontologia sono:
- FullTime
- Idependent
- PartTime
- Retired
- Student
- Unemployed
---
### ProgrammingLanguage
Linguaggi interpretabili da un computer usati per scrivere software. Sono incluse categorie di linguaggi non strettamente di programmazione ma adiacenti, come linguaggi di markup e di query.
Le istanze di questa classe sono collegate a dbpedia con owl:sameAs.
---
### OperatingSystemFamily
Rappresenta le quattro categorie in cui è possibile classificare ogni sistema operativo. Le sue istanze definite insieme all'ontologia sono:
- LinuxBased
- MacOS
- Windows
- OtherOS
---



# Parte 1 - Unificazione dei dati

## Data cleaning
Nonostante i dati in sè siano di buona qualità, il formato delle domande e delle risposte non è costante da anno ad anno, quindi prima di raccogliere i dati in un grafo RDF ho avuto bisogno di uniformarli.

Tra gli elementi che variano da anno ad anno abbiamo:
- Il formato del campo dell'età. Alcune volte è lasciato inserire all'utente come numero floating point, mentre altre viene selezionato da dei range prestabiliti. Nel primo caso ho fatto coincidere i valori di age_min e age_max.
- Alcuni sondaggi fanno una distinzione tra sistemi operativi usati in ambito personale o professionale mentre altri no. In tal caso ho unito le due categorie, inoltre solo a partire dal 2022 è possibile sceglierne più di uno.
- Il wording delle risposte a scelta multipla non è quasi mai consistente e le opzioni vanno e vengono. Per questi campi ho definito io delle categorie in cui smistare le risposte in base a delle parole chiave.
- Anche la selezione stessa di linguaggi e sistemi operativi non è costante ma cambia da anno ad anno.

A seguire definisco una serie di funzioni che mi permettono di eseguire questi ed altri aggiustamenti.

In [1]:
pip install pandas numpy

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------------------------------- ------ 51.2/61.0 kB 890.4 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 819.4 kB/s eta 0:00:00
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.5 MB 4.5 MB/s eta 0:00:03
   ---------------------------------------- 0.1/11.5 MB 2.1 MB/s eta 0:00:06
    --------------------------------------- 0.2/11.5 MB 2.4 MB/s eta 0:00:05
    --------------------------------------- 0.3/11.5 MB 1.9 MB/s eta 0:00:06
   - -------------------------------------- 0.3/11.5 MB 1.9 MB/s eta 0:00:06
   - ------------------------

In [2]:
import pandas as pd
import numpy as np

In [164]:
#lettura dei dataset
surveys = [pd.read_csv('raw/'+ str(year) + ".csv", low_memory=False, index_col=0) for year in range(2023, 2019,-1)]

---
### Funzioni ausiliarie

Consideriamo le risposte relative al campo di occupazione, e osserviamo come nonostante queste varino tra i diversi sondaggi, basta individuare le parole "learning", "profession" e "primarily" per classificare la risposta come appartenente a uno studente, un developer, o nessuno dei due.

In [156]:
pd.concat(surveys).MainBranch.unique()

array(['None of these', 'I am a developer by profession',
       'I am not primarily a developer, but I write code sometimes as part of my work/studies',
       'I code primarily as a hobby', 'I am learning to code',
       'I used to be a developer by profession, but no longer am',
       'I am not primarily a developer, but I write code sometimes as part of my work',
       'I am a student who is learning to code', nan], dtype=object)

La funzione find_category permette di associare una risposta passata come stringa ad una categoria attraverso la ricerca di un determinate sottostringhe specificate in un dizionario.
Il parametro default indica cosa viene restituito se nessuna chiave è trovata nella stringa. Se omesso restituisce l'input invariato.

In [157]:
def find_category(text, keywords, default=None):
    if pd.isna(text): return np.nan
    low_text = text.lower()

    for key in keywords.keys():
        if key in low_text: return keywords[key]

    if default is None: return text
    else: return default


Con lo stesso approccio diventa possibile unificare le risposte variabili nei campi occupazione, sistemi operativi, e linguaggi.

In [158]:

branch_conversion = {"learning": "LearningToCode",
        "profession": "Developer",
        "primarily": "NonDeveloper"}

employment_conversion ={'full-time':'FullTime',
        'independent':'Independent',
        'student':'Student',
        'part-time':'PartTime',
        'not employed':'Unemployed',
        'retired':'Retired'}

os_conversion = {'windows':'Windows',
                      'macos':'MacOS',
                      'android':'OtherOS',
                      'ios':'OtherOS',
                      'ipados':'OtherOS',
                      'bsd':'OtherOS',
                      'haiku':'OtherOS',
                      'other':'OtherOS'}

lang_conversion = {'html':'HTML',
                'bash':'Bash',
                'visual basic':'Visual Basic .Net'}

Esempi di esecuzione, la priorità è data alla keyword specificata per prima nel dizionario.

In [159]:
txtA = "Esempio A - By profession"
txtB = "Esempio B - primarily profession learning"
txtC = "Esempio C - nessuno"

keys = branch_conversion

print("-- "+txtA)
print(find_category(txtA, keys))
print("-- "+txtB)
print(find_category(txtB, keys))
print("-- "+txtC)
print(find_category(txtC, keys))
print(find_category(txtC, keys, np.nan))

Esempio A - By profession
Developer
Esempio B - primarily profession learning
LearningToCode
Esempio C - nessuno
Esempio C - nessuno
nan


I campi sistemi operativi e lingue permettono più risposte per utente, quindi ho bisogno anche di una funzione che applichi find_category ad un'intera lista.

In [160]:
def convert_list(lst, conversion, default=None):
    if not isinstance(lst, list): return []
    return [find_category(item, conversion) if default is None else find_category(item, conversion, default) for item in lst]


La funzione get_age_bounds effettua il parsing delle stringhe contenenti un range di età in formato "xx-yy years old" e restituisce una coppia di interi contenente il minimo e il massimo del range.

In [161]:
import re

def get_age_bounds(age):
    if pd.isna(age): return (np.nan, np.nan)
    bounds = re.findall(r'\d\d', age)

    if len(bounds) == 1:
        limit = int(bounds[0])
        if limit <= 18: return (np.nan, 17)
        if limit > 64: return (limit, np.nan)

    if len(bounds) == 2: return bounds

    else: return (np.nan, np.nan)

#Esempi
print(get_age_bounds('30-45 years old'))
print(get_age_bounds('Over 65 years old'))
print(get_age_bounds('Under 18'))
print(get_age_bounds('Prefer not to say'))


['30', '45']
(65, nan)
(nan, 17)
(nan, nan)



Per quanto riguarda le nazioni mi avvalgo della libreria [country_converter](https://pypi.org/project/country-converter/), che usa espressioni regolari per identificare la nazione descritta da una stringa e mi permette di ottenerne il nome nella sua forma più comune (essenziale per il linking con dbpedia) e il codice ISO Alpha-3.
Eventuali valori mancanti, paesi non più esistenti e typo vengono trattati come missing values.

---

### Creazione dataframe standardizzati
Usando le funzioni appena discusse, i dati corrispondenti ai sondaggi dei quattro anni considerati vengono organizzati in quattro dataframe con colonne:
**['branch', 'country', 'ISO3', 'age_min', 'age_max', 'employment', 'OS', 'langs']**

In [165]:
import country_converter as coco
cc = coco.CountryConverter()

processed_surveys = []

for year in range(4):

    #Struttura del dataframe lavorato
    s = pd.DataFrame(columns=['branch', 'country', 'ISO3', 'age_min', 'age_max', 'employment', 'OS', 'langs'])


    #TIPO DI UTENTE
    s.branch = surveys[year].MainBranch.apply(lambda x: find_category(x, branch_conversion, default=np.nan))
    s.employment = surveys[year].Employment.str.split(';').str.get(0).str.replace(',','').apply(lambda x: find_category(x, employment_conversion, default=np.nan))


    #NAZIONI
    s.country = surveys[year].Country.apply(lambda name: np.nan if pd.isna(name) or name=='Nomadic' else cc.convert(name, to = 'name_short'))
    s.country = s.country.replace("not found", np.nan)
    s.ISO3 = s.country.apply(lambda name: np.nan if pd.isna(name) or name=='Nomadic' else cc.convert(name, to = 'ISO3'))
    s.ISO3 = s.ISO3.replace("not found", np.nan)


    #ETà
    if year > 2: #2020
        s.age_max = surveys[year].Age
        s.age_min = s.age_max
    else:       #2021/22/23
        s['age_bounds'] = surveys[year].Age.apply(get_age_bounds) if year < 3 else surveys[year].Age.apply(lambda x: (x, x))
        s[['age_min', 'age_max']] = s['age_bounds'].apply(lambda x: pd.Series(x))
        s = s.drop('age_bounds', axis = 1)



    #SITEMI OPERATIVI
    if year<=1: #2023 e 2022
        personal = surveys[year]['OpSysPersonal use'].str.split(';').apply(lambda l: convert_list(l, os_conversion, default='LinuxBased'))
        professional = surveys[year]['OpSysProfessional use'].str.split(';').apply(lambda l: convert_list(l, os_conversion, default='LinuxBased'))
        s.OS = pd.concat([personal, professional], axis=1).apply(lambda row: list(set(row[0]+row[1])), axis=1)
        
    else: #2021 e 2020
        s.OS = surveys[year].OpSys.apply(lambda item: find_category(item, os_conversion, default='LinuxBased'))
        s.OS = s.OS.apply(lambda x: [x] if pd.notna(x) else [])


    #LINGUAGGI
    lang_column = 'LanguageHaveWorkedWith' if year<3 else 'LanguageWorkedWith'
    s.langs = surveys[year][lang_column].str.split(';').apply(lambda l: convert_list(l, lang_conversion))


    #drop di tutte le righe completamente vuote
    s.dropna(how='all', inplace=True)
    s.reset_index(inplace=True, drop=True)
    
    processed_surveys.append(s)
    print("Processato il sondaggio "+str(2023-year))




Processato il sondaggio 2023
Processato il sondaggio 2022
Processato il sondaggio 2021
Processato il sondaggio 2020


In [166]:
processed_surveys

[             branch        country ISO3 age_min age_max employment  \
 0               NaN            NaN  NaN      18      24        NaN   
 1         Developer  United States  USA      25      34   FullTime   
 2         Developer  United States  USA      45      54   FullTime   
 3         Developer  United States  USA      25      34   FullTime   
 4         Developer    Philippines  PHL      25      34   FullTime   
 ...             ...            ...  ...     ...     ...        ...   
 89179     Developer         Brazil  BRA      25      34   FullTime   
 89180     Developer        Romania  ROU      18      24   FullTime   
 89181  NonDeveloper         Israel  ISR     NaN     NaN        NaN   
 89182     Developer    Switzerland  CHE     NaN    17.0   PartTime   
 89183     Developer           Iran  IRN      35      44   FullTime   
 
                                           OS  \
 0                                         []   
 1                  [Windows, MacOS, OtherOS]   

In [68]:
for y in range(4):
    processed_surveys[y].to_csv("processed/Sondaggio "+ str(2023-y) + " lavorato.csv")

NameError: name 'processed_surveys' is not defined

---

# Parte 2 - RDF e linking
Adesso procedo a usare le tabelle create nella parte 1 per popolare un grafo RDF.

In [4]:
import pandas as pd
import numpy as np
from ast import literal_eval
from urllib.parse import quote, unquote
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, OWL

In [5]:
#Lettura DF ordinati
processed_surveys = [pd.read_csv('processed/Sondaggio '+ str(year) + " lavorato.csv", low_memory=False, index_col=0) for year in range(2023, 2019,-1)]

Urify trasforma una stringa in una URI valida.

In [6]:

def urify(name):
    return quote(str(name).replace(" ","_").replace("?",""))

def deurify(uri):
    return unquote(str(uri).replace("_"," ").replace("","?"))

Per distinguere gli elementi dell'ontologia dalle istanze di dati uso due prefissi diversi: **SDO** e **SDR**.

In [24]:

#Prefissi
base_uri = 'http://www.so.developersurvey.org/resource/'
ontology_uri = 'http://www.so.developersurvey.org/ontology/'
dbpedia = 'http://www.dbpedia.org/resource/'

g=Graph()

#Namespaces
sdo = Namespace(ontology_uri)
g.bind('sdo', sdo)
sdr = Namespace(base_uri)
g.bind('sdr', sdr)
dbr = Namespace(dbpedia)
g.bind('dbr', dbr)



---
## Linking dei linguaggi
Tradurre ogni linguaggio di programmazione in una uri dbpedia valida si è rivelato un compito molto complesso, dato che molti nomi portano a delle disambiguazioni, e non c'è buona coerenza tra le classi dbo a cui appartengono le loro pagine.
Fortunatamente, però, sono riuscito a trovare [un dataset](https://www.back4app.com/database/paul-datasets/list-of-all-programming-languages/dataset) pubblico che mette in relazione i nomi di centinaia di linguaggi con i corrispettivi link di Wikipedia.

Dato che il dataset non proviene da una fonte altamente affidabile, prima di procedere ho ritenuto opportuno verificare che fosse corretto quanto meno per le lingue presenti nei miei dati. Perciò, dopo avere estratto i nomi dalle url Wikipedia ho effettuato delle query sparql per confermare l'esistenza di una pagina dbpedia per ogni linguaggio di programmazione menzionato nei sondaggi, e ne ho controllato il contenuto.

In [8]:
lang_uris = pd.read_csv('raw/all_programming_languages.csv', low_memory=False)
lang_uris.ProgrammingLanguage = lang_uris.ProgrammingLanguage.apply(lambda x: x.lower())
lang_uris = lang_uris.set_index('ProgrammingLanguage')['Source'].str.split('/').str.get(-1)

In [68]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

all_surveys = pd.concat(processed_surveys, ignore_index=True)
all_langs = all_surveys.langs.apply(literal_eval).explode().dropna().apply(lambda x: x.lower()).unique()

for item in all_langs:
    uri = dbpedia+lang_uris.loc[item]

    sparql.setQuery("""
        ASK{
            VALUES (?r) {(<%s>)}
             ?r [] [].
        }
    """%unquote(uri))

    try:
        if sparql.query().convert()['boolean']:
            print(item+': '+uri)

        else:
            print(item + ' -- Not found!')
    except:
        print(item + ' -- Bad request!')



html: http://dbpedia.org/resource/HTML
javascript: http://dbpedia.org/resource/JavaScript
python: http://dbpedia.org/resource/Python_(programming_language)
shell: http://dbpedia.org/resource/Shell_(computing)
go: http://dbpedia.org/resource/Go_(programming_language)
php: http://dbpedia.org/resource/PHP
ruby: http://dbpedia.org/resource/Ruby_(programming_language)
sql: http://dbpedia.org/resource/SQL
typescript: http://dbpedia.org/resource/TypeScript
ada: http://dbpedia.org/resource/Ada_(programming_language)
clojure: http://dbpedia.org/resource/Clojure
elixir: http://dbpedia.org/resource/Elixir_(programming_language)
java: http://dbpedia.org/resource/Java_(programming_language)
lisp: http://dbpedia.org/resource/Lisp_(programming_language)
ocaml: http://dbpedia.org/resource/OCaml
raku: http://dbpedia.org/resource/Raku_(programming_language)
scala: http://dbpedia.org/resource/Scala_(programming_language)
swift: http://dbpedia.org/resource/Swift_(programming_language)
zig: http://dbpedia.

## Linking delle nazioni

Successivamente applico lo stesso controllo alle nazioni ottenute da country_converter.

In [10]:

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)


all_surveys = pd.concat(processed_surveys, ignore_index=True)
all_countries = all_surveys.country.dropna().unique()


for item in all_countries:
    name = unquote(urify(item))
    sparql.setQuery("""
        ASK
        {
            dbr:%s [] [].
        }
    """%name)

    try:
        if sparql.query().convert()['boolean']:
            print(name+ ': ' + dbpedia + name)

        else:
            print(name + ' -- Not found!')
    except:
        print(name + ' -- Bad request!')



United_States: http://www.dbpedia.org/resource/United_States
Philippines: http://www.dbpedia.org/resource/Philippines
United_Kingdom: http://www.dbpedia.org/resource/United_Kingdom
Finland: http://www.dbpedia.org/resource/Finland
India: http://www.dbpedia.org/resource/India
Australia: http://www.dbpedia.org/resource/Australia
Netherlands: http://www.dbpedia.org/resource/Netherlands
Germany: http://www.dbpedia.org/resource/Germany
Sweden: http://www.dbpedia.org/resource/Sweden
France: http://www.dbpedia.org/resource/France
Albania: http://www.dbpedia.org/resource/Albania
Nigeria: http://www.dbpedia.org/resource/Nigeria
Spain: http://www.dbpedia.org/resource/Spain
South_Africa: http://www.dbpedia.org/resource/South_Africa
Belgium: http://www.dbpedia.org/resource/Belgium
Italy: http://www.dbpedia.org/resource/Italy
Brazil: http://www.dbpedia.org/resource/Brazil
Portugal: http://www.dbpedia.org/resource/Portugal
Bangladesh: http://www.dbpedia.org/resource/Bangladesh
Canada: http://www.dbpe

A questo punto posso già aggiungere le nazioni al grafo, effettuando il collegamento agli oggetti equivalenti di dbpedia con la object property owl:sameAs.

In [25]:
import country_converter as coco
cc = coco.CountryConverter()

all_surveys = pd.concat(processed_surveys, ignore_index=True)
all_countries = all_surveys.country.dropna().unique()
all_countries = pd.Series(all_countries)
all_iso = all_countries.apply(lambda x: cc.convert(x, to="ISO3"))
all_country_codes = pd.DataFrame({'Country':all_countries, 'ISO3':all_iso})

for id, row in all_country_codes.iterrows():
    country_uri = base_uri+urify(row.Country)
    g.add([URIRef(country_uri), URIRef(RDF.type), URIRef(sdo.Country)])
    g.add([URIRef(country_uri), URIRef(RDFS.label), Literal(row.Country)])
    g.add([URIRef(country_uri), URIRef(OWL.sameAs), URIRef(dbpedia+urify(row.Country))])
    g.add([URIRef(country_uri), URIRef(sdo.ISO_A3), Literal(row.ISO3)])

---
## Popolazione grafo
Adesso è il turno dei dati relativi ai sondaggi specifici.
add_survey prende in input uno dei dataframe ottenuti nella fase 1 e l'anno di riferimento e li usa per popolare il grafo.
- Inizialmente aggiunge l'istanza relativa al sondaggio in generale e le relative proprietà.
- Successivamente aggiunge i linguaggi di programmazione, e usando il dataset degli uri già menzionato li collega con dbpedia.
- Infine per ogni riga del dataframe aggiunge le triple corrispondenti al singolo utente.

Il motivo per cui i linguaggi sono aggiunti adesso e non a parte sta nel fatto che li associo ai sondaggi dove compaiono come opzione.
Questo causa della ridondanza quando una lingua è presente in più sondaggi e fa in modo che il programma tenti di reinserire delle triple uguali, ma si tratta di pochi elementi che non hanno un grande impatto sul tempo di esecuzione.

In [26]:
def add_survey(survey, year):
    #survey = survey.head(5)

    #aggiunta sondaggi
    survey_uri = base_uri+'survey_'+str(year)
    g.add([URIRef(survey_uri), URIRef(RDF.type), URIRef(sdo.StackOverflowSurvey)])
    g.add([URIRef(survey_uri), URIRef(sdo.year), Literal(year)])
    g.add([URIRef(survey_uri), URIRef(sdo.totalResponses), Literal(len(survey.index))])
    
    
    #aggiunta linguaggi
    for lang in survey.langs.apply(literal_eval).explode().dropna().unique():
        my_lang_uri = base_uri+urify(lang)
        g.add([URIRef(my_lang_uri), URIRef(RDF.type), URIRef(sdo.ProgrammingLanguage)])
        g.add([URIRef(my_lang_uri), URIRef(RDFS.label), Literal(lang)])
        g.add([URIRef(my_lang_uri), URIRef(sdo.isOptionInSurvey), URIRef(survey_uri)])
        g.add([URIRef(survey_uri), URIRef(sdo.hasLanguageOption), URIRef(my_lang_uri)])
        g.add([URIRef(my_lang_uri), URIRef(OWL.sameAs), URIRef(dbpedia+urify(lang_uris.loc[lang.lower()]))])

    #aggiunta risposte
    for ind, row in survey.iterrows():
        row_uri = base_uri+str(year)+'_'+str(ind)
        
        #branch
        row_type = sdo.SurveyRespondant
        if not pd.isna(row.branch):row_type = ontology_uri+row.branch
        g.add([URIRef(row_uri), URIRef(RDF.type), URIRef(row_type)])
        g.add([URIRef(row_uri), URIRef(sdo.isFromSurvey), URIRef(survey_uri)])
        
        #country
        if not pd.isna(row.country):
            g.add([URIRef(row_uri), URIRef(sdo.isFromCountry), URIRef(base_uri+urify(row.country))])
        
        #age
        if not pd.isna(row.age_min):
            g.add([URIRef(row_uri), URIRef(sdo.minAge), Literal(int(row.age_min))])

        if not pd.isna(row.age_max):
            g.add([URIRef(row_uri), URIRef(sdo.maxAge), Literal(int(row.age_max))])

        
        #employment
        if not pd.isna(row.employment):
            g.add([URIRef(row_uri), URIRef(sdo.hasEmployment), URIRef(ontology_uri+row.employment)])
        
        #os
        if len(row.OS)>0:
            for os in literal_eval(row.OS):
                g.add([URIRef(row_uri), URIRef(sdo.usesOS), URIRef(ontology_uri+os)])

        #language
        if len(row.langs)>0:
            for l in literal_eval(row.langs):
                g.add([URIRef(row_uri), URIRef(sdo.usesLanguage), URIRef(base_uri+urify(l))])


In [27]:
for n in range(4):
    add_survey(processed_surveys[n], 2023-n)
    print(str(2023-n) + " Was added to the graph.")

2023 Was added to the graph.
2022 Was added to the graph.
2021 Was added to the graph.
2020 Was added to the graph.


Concludo con la serializzazione del grafo in formato turtle

In [28]:
g.serialize(destination='RDF/surveys_rdf_data.ttl',
            format='turtle')

<Graph identifier=N39d64e624a9a4abab756ce21e7b78822 (<class 'rdflib.graph.Graph'>)>